# Airbnb Web Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import json
import time

import os

import pandas as pd

from multiprocessing import Pool

In [2]:
link = "https://www.airbnb.fr/s/Paris/homes?place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&refinement_paths%5B%5D=%2Fhomes&search_type=filter_change&tab_id=home_tab&query=Paris&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&checkin=2022-04-07&checkout=2022-04-10&adults=2&source=structured_search_input_header"

## 1. Scraping search page

In [3]:
def scrape_page(page_url):
    """Extracts HTML from a webpage"""
    
    answer = requests.get(page_url)
    content = answer.content
    soup = BeautifulSoup(content, features='html.parser')
    
    return soup

In [4]:
def extract_listing(page_url):
    """Extracts listings from an Airbnb search page"""
    
    page_soup = scrape_page(page_url)
    listings = page_soup.findAll("div", {"class": "_gig1e7"})

    return listings

In [24]:
FEATURE_DICT = {
    'name':["div", {'class':'mj1p6c8 dir dir-ltr'}],
    'main_name': ["div", {'class':'c1bx80b8 dir dir-ltr'}],
    'price': ["div", {'class':'p1qe1cgb dir dir-ltr'}],
    'label': ["div", {'class':'s1sphtog dir dir-ltr'}],
    'rating': ["div", {'class':'sglmc5a dir dir-ltr'}]
}
attributes = ["span", {'class':'mp2hv9t dir dir-ltr'}]

In [25]:
test = pd.DataFrame(FEATURE_DICT)
test

,name,main_name,price,label,rating
0,div,div,div,div,div
1,{'class': 'mj1p6c8 dir dir-ltr'},{'class': 'c1bx80b8 dir dir-ltr'},{'class': 'p1qe1cgb dir dir-ltr'},{'class': 's1sphtog dir dir-ltr'},{'class': 'sglmc5a dir dir-ltr'}


In [26]:
listing_soups = extract_listing(link)
columns = ['name',
          'main_name',
          'price',
          'label',
          'rating',
          'attributes']
df = pd.DataFrame(columns)
for element_soup in listing_soups:
    result = {}
    for feature in FEATURE_DICT:
        try:
            result[feature] = element_soup.findAll(FEATURE_DICT[feature][0])[0].text
        except:
            result['name'] = None
        try:
            result['attributes'] = [element_soup.findAll(attributes[0])[i].text for i in range(len(element_soup.findAll(attributes[0])))]
        except:
            result['attributes'] = None
        df_temp = pd.DataFrame(result)
        df = pd.concat([df, df_temp])

df

,0,attributes,name,main_name,price,label,rating
0,name,NaN,NaN,NaN,NaN,NaN,NaN
1,main_name,NaN,NaN,NaN,NaN,NaN,NaN
2,price,NaN,NaN,NaN,NaN,NaN,NaN
3,label,NaN,NaN,NaN,NaN,NaN,NaN
4,rating,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
36,NaN,/ nuit,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...
37,NaN,135€ par nuit,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...
38,NaN,405€ au total,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...
39,NaN,405€ au total,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...,Logement entier : appartement · Particulier · ...


In [10]:
soup = scrape_page(link)
name = soup.findAll("div", {'class':'mj1p6c8 dir dir-ltr'})
name

[<div class="mj1p6c8 dir dir-ltr">Logement entier dans une résidence de tourisme<span aria-hidden="true"> · </span>Professionnel<span aria-hidden="true"> · </span>13e arrondissement</div>,
 <div class="mj1p6c8 dir dir-ltr">Appartement entier<span aria-hidden="true"> · </span>Professionnel<span aria-hidden="true"> · </span>20e arrondissement</div>,
 <div class="mj1p6c8 dir dir-ltr">Logement entier : appartement<span aria-hidden="true"> · </span>Particulier<span aria-hidden="true"> · </span>14e arrondissement</div>,
 <div class="mj1p6c8 dir dir-ltr">Logement entier : appartement en résidence<span aria-hidden="true"> · </span>Professionnel<span aria-hidden="true"> · </span>Asnières-sur-Seine</div>,
 <div class="mj1p6c8 dir dir-ltr">Chambre privée<span aria-hidden="true"> · </span>Professionnel<span aria-hidden="true"> · </span>16e arrondissement</div>,
 <div class="mj1p6c8 dir dir-ltr">Logement entier : appartement<span aria-hidden="true"> · </span>Professionnel<span aria-hidden="true"> ·

In [6]:
name[0].text

'Logement entier dans une résidence de tourisme · Professionnel · 13e arrondissement'

In [ ]:
RULES_SEARCH_PAGE = {
    'url': {'tag': 'a', 'get': 'href'},
    'name': {'tag': 'div', 'class': '_hxt6u1e', 'get': 'aria-label'},
    'name_alt': {'tag': 'a', 'get': 'aria-label'},
    'header': {'tag': 'div', 'class': '_b14dlit'},
    'rooms': {'tag': 'div', 'class': '_kqh46o'},
    'facilities': {'tag': 'div', 'class': '_kqh46o', 'order': 1},
    'badge': {'tag': 'div', 'class': '_17bkx6k'},
    'rating_n_reviews': {'tag': 'span', 'class': '_18khxk1'},
    'price': {'tag': 'span', 'class': '_1p7iugi'},
    'price_alt': {'tag': 'span', 'class': '_olc9rf0'},
    'superhost': {'tag': 'div', 'class': '_ufoy4t'},
}

In [36]:

listing_soups = extract_listing(link)
test = {
    'name': listing_soups[0].findAll("div", {'class':'mj1p6c8 dir dir-ltr'})[0].text,
    'main_name': listing_soups[0].findAll("div", {'class':'c1bx80b8 dir dir-ltr'})[0].text,
    'attributes': [[listing_soups[0].findAll("span", {'class':'mp2hv9t dir dir-ltr'})[i].text for i in range(len(listing_soups[0].findAll("span", {'class':'mp2hv9t dir dir-ltr'})))]],
    'price': listing_soups[0].findAll("div", {'class':'p1qe1cgb dir dir-ltr'})[0].text,
    'label': listing_soups[0].findAll("div", {'class':'s1sphtog dir dir-ltr'})[0].text,
    'rating': listing_soups[0].findAll("div", {'class':'sglmc5a dir dir-ltr'})[0].text
}

#name in grey

#name in black

test
test = pd.DataFrame(test)
test['attributes']

    

0    [2 voyageurs, Studio, 1 lit, 1 salle de bain, ...
Name: attributes, dtype: object

In [38]:
test['attributes'][0][0]

'2 voyageurs'

In [31]:
l = ['a', 'b']
array()

"['a', 'b']"

In [16]:
#Price
listing_soups[0].findAll("div", {'class':'p1qe1cgb dir dir-ltr'})[0].text

'76€\xa0/ nuit76€ par nuit227€ au total227€ au totalAfficher le décompte du prix'

In [21]:
#Label
listing_soups[0].findAll("div", {'class':'s1sphtog dir dir-ltr'})[0].text


'Perle rare,&nbsp;\xa0·\xa0Cet endroit est généralement réservé.'

In [27]:
#Ratings
listing_soups[0].findAll("div", {'class':'sglmc5a dir dir-ltr'})[0].text


'4,22\xa0(1\u202f022 commentaires)'